In [1]:
# This cell imports stuff, and sets up a bot instance etc

import sys
import lzma
from s2clientprotocol.sc2api_pb2 import Response, ResponseObservation
from MapAnalyzer.MapData import MapData
import pickle
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("../src")
sys.path.append("../tests")
sys.path.append("../src/ares")
from sc2.position import Point2
from sc2.client import Client
from sc2.game_data import GameData
from sc2.game_info import GameInfo
from sc2.game_state import GameState
from unittest.mock import patch
from ares import AresBot
from ares.dicts.unit_data import UNIT_DATA
from sc2.bot_ai import BotAI

# load AresBot and MapAnalyzer object instances from pickled files
from tests.load_bot_from_pickle import build_bot_object_from_pickle_data
GRESVAN = "../tests/pickle_data/GresvanAIE.xz"
bot = await build_bot_object_from_pickle_data(GRESVAN)
data = MapData(bot)

# common variables
grid = data.get_pyastar_grid()
position = bot.enemy_start_locations[0]
units = bot.all_units

%load_ext line_profiler
%load_ext Cython

C:\Users\Tom\AppData\Local\pypoetry\Cache\virtualenvs\ares-sc2-S437wHWG-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-21 19:48:16.818 | INFO     | MapAnalyzer.MapData:__init__:122 - dev Compiling GresvanAIE 
 Version dev Map Compilation Progress : 0.4it [00:00,  1.23it/s]
2023-08-21 19:48:17.226 | INFO     | ares.managers.placement_manager:initialise:175 - Solved placement formation in 13.519048690795898 ms
2023-08-21 19:48:17.397 | INFO     | MapAnalyzer.MapData:__init__:122 - dev Compiling GresvanAIE 
 Version dev Map Compilation Progress : 0.4it [00:00,  1.21it/s]


# Cythonizing commonly used functions in a python-sc2 bot

Actual speedups many vary as notebook gets rerun

* [Converting is_position_safe](#Python-version-of-is_position_safe) **6.57x speedup**
* [Alternative to python-sc2's units.closest_to](#Alternative-to-python-sc2's-units.closest_to) **6.7x speedup**
* [Speeding up `Units.center`](#Speeding-up-Units.center) **2.01x speedup**
* [Distance to / `unit.distance_to`](#Distance-to-/-unit.distance_to) **2.47x speedup**
* [Convert `already_pending` for units](#Convert-already_pending-for-units) **6.62x speedup**
* [Alternative to `python-sc2`'s `units.in_attack_range`](#Alternative-to-python-sc2's-units.in_attack_range) **4.17x speedup**
* [Calculate if attack is ready](#Calculate-if-attack-is-ready) **3.87x speedup**
* [Pick enemy target](#Pick-enemy-target) **1.63x speedup**
* [Converting `units.sorted_by_distance_to`](#Converting-units.sorted_by_distance_to) **7.3x speedup**
* [Convert `unit.is_facing()`](#Convert-unit.is_facing()) **9.1x speedup**
* [Convert `Point2.towards()`](#Convert-Point2.towards()) **14.29x speedup**

# Converting `is_position_safe` to cython

## Python version of `is_position_safe`

In [2]:
def is_position_safe(
    grid: np.ndarray,
    position: Point2,
    weight_safety_limit: float = 1.0,
) -> bool:
    weight: float = grid[position.rounded]
    # np.inf check if drone is pathing near a spore crawler
    return weight == np.inf or weight <= weight_safety_limit

In [3]:
%timeit is_position_safe(grid, position)

2.3 µs ± 17.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [4]:
%lprun -f is_position_safe is_position_safe(grid, position)

Timer unit: 1e-07 s

Total time: 4.81e-05 s
File: C:\Users\Tom\AppData\Local\Temp\ipykernel_23956\2777119042.py
Function: is_position_safe at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def is_position_safe(
     2                                               grid: np.ndarray,
     3                                               position: Point2,
     4                                               weight_safety_limit: float = 1.0,
     5                                           ) -> bool:
     6         1        174.0    174.0     36.2      weight: float = grid[position.rounded]
     7                                               # np.inf check if drone is pathing near a spore crawler
     8         1        307.0    307.0     63.8      return weight == np.inf or weight <= weight_safety_limit

## Cython version of `is_position_safe`

In [5]:
%%cython
import numpy as np
cimport numpy as cnp
from cython cimport boundscheck, wraparound
@boundscheck(False)
@wraparound(False)
cpdef bint is_position_safe(
    cnp.ndarray[cnp.npy_float32, ndim=2] grid,
    (unsigned int, unsigned int) position,
    double weight_safety_limit = 1.0,
):
    cdef double weight = 0.0
    weight = grid[position[0], position[1]]
    # np.inf check if drone is pathing near a spore crawler
    return weight == np.inf or weight <= weight_safety_limit

In [6]:
%timeit is_position_safe(grid, position.rounded)

612 ns ± 2.97 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Alternative to `python-sc2`'s `units.closest_to`

In [7]:
units = bot.all_units
position = bot.enemy_start_locations[0]
unit = units[0]

In [8]:
# slower using closest_to(Point2)
%timeit units.closest_to(position)

242 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
# this is faster since distance between all units is cached
%timeit units.closest_to(unit)

125 µs ± 918 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
%%cython
from cython cimport boundscheck, wraparound

cdef double euclidean_distance_squared(
        (float, float) p1,
        (float, float) p2
):
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2

@boundscheck(False)
@wraparound(False)
cpdef object closest_to((float, float) position, object units):
    cdef:
        object closest = units[0]
        double closest_dist = 999.9
        double dist = 0.0
        unsigned int len_units = len(units)
        (float, float) pos
        
    for i in range(len_units):
        unit = units[i]
        pos = unit.position
        dist = euclidean_distance_squared((pos[0], pos[1]), (position[0], position[1]))
        if dist < closest_dist:
            closest_dist = dist
            closest = unit
            
    return closest


In [11]:
%timeit closest_to(position, units)

16.4 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Speeding up `Units.center`

Similar to `units.center` in `python-sc2`, but tweaked to work in this notebook

In [12]:
from sc2.position import Point2
def center(units) -> Point2:
    """Returns the central position of all units."""
    assert units, f"Units object is empty"
    amount = units.amount
    return Point2(
        (
            sum(unit._proto.pos.x for unit in units) / amount,
            sum(unit._proto.pos.y for unit in units) / amount,
        )
    )

In [13]:
%timeit center(units)

134 µs ± 515 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Convert `Units.center` to cython

In [14]:
%%cython

cimport cython
from sc2.position import Point2

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef (double, double) cy_center(object units):
    """Returns the central position of all units."""
    cdef:
        unsigned int i = 0
        unsigned int num_units = len(units)
        double sum_x, sum_y = 0.0
        (double, double) position
        object unit

    for i in range(num_units):
        pos = units[i]._proto.pos
        position = (pos.x, pos.y)
        sum_x += position[0]
        sum_y += position[1]

    return (sum_x / num_units, sum_y / num_units)

In [15]:
%timeit Point2(cy_center(units))

65.3 µs ± 278 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Distance to / `unit.distance_to`

Check and profile distances between 2 units / 1 unit and a Point2 / two Point2's using `python-sc2` implementation of `distance_to`. 

Then see if cython does it faster

In [16]:
unit1 = bot.workers[0]
unit2 = bot.workers[4]
position1 = bot.game_info.map_center
position2 = bot.main_base_ramp.top_center

In [17]:
unit1.distance_to(unit2)

6.0901554167386

In [18]:
%timeit unit1.distance_to(unit2)

620 ns ± 6.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [19]:
unit1.distance_to(position1)

75.3884622135151

In [20]:
%timeit unit1.distance_to(position1)

1.04 µs ± 5.32 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
position1.distance_to(position2)

66.7720001198107

In [22]:
%timeit position1.distance_to(position2)

383 ns ± 1.77 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Convert `distance_to` to cython

In [23]:
%%cython

cpdef double cy_distance_to(
        (float, float) p1,
        (float, float) p2
):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

In [24]:
cy_distance_to(unit1.position, unit2.position)

6.090155377590407

In [25]:
%timeit cy_distance_to(unit1.position, unit2.position)

200 ns ± 1.11 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [26]:
cy_distance_to(unit1.position, position1)

75.38846337574635

In [27]:
%timeit cy_distance_to(unit1.position, position1)

169 ns ± 0.74 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [28]:
cy_distance_to(position1, position2)

66.7720001198107

In [29]:
%timeit cy_distance_to(position1, position2)

156 ns ± 0.755 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


# Convert `already_pending` for units

This seems like a slow function in profiles, make an alternative for units and structures

## `python-sc2`'s `already_pending` function:

In [30]:
from sc2.ids.unit_typeid import UnitTypeId

In [31]:
%timeit bot.already_pending(UnitTypeId.MARINE)

2.61 µs ± 10.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Making a python alternative that's already more than 2x faster

In [32]:
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.game_info import Race
from ares.dicts.does_not_use_larva import DOES_NOT_USE_LARVA

In [33]:
def unit_pending(unit_type: UnitTypeId) -> int:
    trained_from = UNIT_TRAINED_FROM[unit_type]
    if bot.race != Race.Zerg or unit_type == UnitTypeId.QUEEN:
        return len(
            [
                s
                for s in bot.structures
                if s.orders and s.type_id in trained_from
                and s.orders[0].ability.button_name.upper() == UnitTypeId.MARINE.name
            ]
        )
    # check eggs and cocoons for Zerg
    else:
        if type_id in DOES_NOT_USE_LARVA:
            if type_id == UnitTypeId.LURKERMP:
                return len(bot.own_units(UnitTypeId.LURKERMPEGG))
            elif type_id == UnitTypeId.OVERSEER:
                return len(bot.own_units(UnitTypeId.OVERLORDCOCOON))
            return len(bot.own_units(UnitTypeId[f"{type_id.name}COCOON"]))
        else:
            return len(
                [
                    egg
                    for egg in self.eggs
                    if egg.orders[0].ability.button_name.upper() == type_id.name
                ]
            )

In [34]:
%timeit unit_pending(UnitTypeId.MARINE)

1.18 µs ± 4.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [35]:
%%cython

from sc2.ids.unit_typeid import UnitTypeId
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.game_info import Race
from ares.dicts.does_not_use_larva import DOES_NOT_USE_LARVA

cimport cython

cpdef unsigned int cy_unit_pending(object unit_type, object bot):
    cdef:
        unsigned int num_pending = 0
        unsigned int len_units, x
        set trained_from = UNIT_TRAINED_FROM[unit_type]
        object units_collection, unit

    if bot.race == Race.Zerg and unit_type != UnitTypeId.QUEEN:
        if unit_type in DOES_NOT_USE_LARVA:
            units_collection = bot.own_units
            len_units = len(bot.own_units)
            trained_from = {UnitTypeId[f"{unit_type.name}COCOON"]}
            if unit_type == UnitTypeId.LURKERMP:
                trained_from = {UnitTypeId.LURKERMPEGG}
            elif unit_type == UnitTypeId.OVERSEER:
                trained_from = {UnitTypeId.OVERLORDCOCOON}
            
            for x in range(len_units):
                unit = units_collection[x]
                if unit.type_id in trained_from:
                    num_pending += 1
            return num_pending
        # unit will be pending in eggs
        else:
            units_collection = bot.eggs
            len_units = len(units_collection)
            for x in range(len_units):
                egg = units_collection[x]
                if egg.orders and egg.orders[0].ability.button_name.upper() == unit_type.name:
                    num_pending += 1
            return num_pending

    # all other units, check the structures they are built from
    else:
        units_collection = bot.structures
        len_units = len(units_collection)
        for x in range(len_units):
            structure = units_collection[x]
            if structure.orders and structure.orders[0].ability.button_name.upper() == unit_type.name:
                num_pending += 1
        return num_pending

In [36]:
%timeit cy_unit_pending(UnitTypeId.MARINE, bot)

416 ns ± 3.42 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Cython version 6-7 times faster with a warning that this doesn't factor in warp ins

Something similar may be done for structures, but for this we rely on a building tracker which is not present in this notebook

# Alternative to `python-sc2`'s `units.in_attack_range`

In [37]:
from sc2.units import Units
_units = Units([u for u in units if not u.is_mineral_field and not u.is_vespene_geyser and u not in bot.destructables], bot)
bigger_units = Units([u for u in units if not u.is_mineral_field and not u.is_vespene_geyser and u not in bot.destructables], bot)
bigger_units.extend(_units)
bigger_units.extend(_units)

In [38]:
unit = _units[-6]

In [39]:
%timeit _units.in_attack_range_of(unit)

66.5 µs ± 659 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [40]:
_units.in_attack_range_of(unit)

[Unit(name='SCV', tag=4348182529)]

In [41]:
%timeit bigger_units.in_attack_range_of(unit)

195 µs ± 389 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [42]:
%%cython

import cython
from sc2.ids.unit_typeid import UnitTypeId
from ares.dicts.unit_data import UNIT_DATA

UNIT_DATA_INT_KEYS = {k.value: v for k, v in UNIT_DATA.items()}

cdef double cy_distance_to(
        (float, float) p1,
        (float, float) p2
):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef list cy_in_attack_range(object unit, object units, double bonus_distance = 0.0):
    if not unit.can_attack:
        return []

    cdef:
        unsigned int x, len_units, type_id_int
        double dist, air_range, ground_range, radius, other_u_radius
        (float, float) unit_pos, other_unit_pos
        bint other_unit_flying, can_shoot_air, can_shoot_ground

    can_shoot_air = unit.can_attack_air
    can_shoot_ground = unit.can_attack_ground
    len_units = len(units)
    returned_units = []
    unit_pos = unit.position
    radius = unit.radius
    air_range = unit.air_range
    ground_range = unit.ground_range

    for x in range(len_units):
        u = units[x]
        type_id_int = unit._proto.unit_type
        unit_data = UNIT_DATA_INT_KEYS.get(type_id_int, None)
        if unit_data:
            other_unit_flying = unit_data["flying"]
            other_unit_pos = u.position
            other_u_radius = u.radius
            dist = cy_distance_to(unit_pos, other_unit_pos)
    
            if can_shoot_air and (other_unit_flying or type_id_int == 4):
                if dist <= air_range + radius + other_u_radius + bonus_distance:
                    returned_units.append(u)
                    # already added, no need to attempt logic below
                    continue
    
            if can_shoot_ground and not other_unit_flying:
                if dist <= ground_range + radius + other_u_radius + bonus_distance:
                    returned_units.append(u)

    return returned_units         

In [43]:
%timeit cy_in_attack_range(unit, _units)

16.5 µs ± 47.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [44]:
cy_in_attack_range(unit, _units)

[Unit(name='SCV', tag=4348182529)]

In [45]:
%timeit cy_in_attack_range(unit, bigger_units)

48.8 µs ± 345 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Calculate if attack is ready

## First python implementation

In [46]:
from sc2.ids.unit_typeid import UnitTypeId as UnitID
from sc2.unit import Unit
from ares.dicts.turn_rate import TURN_RATE
import math

def angle_to(from_pos: Point2, to_pos: Point2) -> float:
    """Angle from point to other point in radians"""
    return math.atan2(to_pos.y - from_pos.y, to_pos.x - to_pos.x)

def angle_diff(a, b) -> float:
    """Absolute angle difference between 2 angles"""
    if a < 0:
        a += math.pi * 2
    if b < 0:
        b += math.pi * 2
    return math.fabs(a - b)

def get_turn_speed(unit) -> float:
    """Returns turn speed of unit in radians"""
    if unit.type_id in TURN_RATE:
        return TURN_RATE[unit.type_id] * 1.4 * math.pi / 180

def range_vs_target(unit, target) -> float:
    """Get the range of a unit to a target."""
    if unit.can_attack_air and target.is_flying:
        return unit.air_range
    else:
        return unit.ground_range

def attack_ready(bot, unit: Unit, target: Unit) -> bool:
    """
    Determine whether the unit can attack the target by the time the unit faces the target.
    Thanks Sasha for writing this out.
    """
    # fix for units, where this method returns False so the unit moves
    # but the attack animation is still active, so the move command cancels the attack
    # need to think of a better fix later, but this is better then a unit not attacking
    # and still better then using simple weapon.cooldown == 0 micro
    type_id = unit.type_id
    if unit.weapon_cooldown > 7 and type_id == UnitID.HYDRALISK:
        return True
    # prevents crash, since unit can't move
    if type_id == UnitID.LURKERMPBURROWED:
        return True
    if not unit.can_attack:
        return False
    # Time elapsed per game step
    step_time: float = bot.client.game_step / 22.4

    # Time it will take for unit to turn to face target
    angle: float = angle_diff(
        unit.facing, angle_to(unit.position, target.position)
    )
    turn_time: float = angle / get_turn_speed(unit)

    # Time it will take for unit to move in range of target
    distance = (
        unit.position.distance_to(target)
        - unit.radius
        - target.radius
        - range_vs_target(unit, target)
    )
    distance = max(0, distance)
    move_time = distance / ((unit.real_speed + 1e-16) * 1.4)

    return step_time + turn_time + move_time >= unit.weapon_cooldown / 22.4

In [47]:
target = cy_in_attack_range(unit, _units)[0]

In [48]:
attack_ready(bot, unit, target)

True

In [49]:
%timeit attack_ready(bot, unit, target)

5.33 µs ± 32.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Work out where any bottlenecks are

In [50]:
%lprun -f attack_ready attack_ready(bot, unit, target)

Timer unit: 1e-07 s

Total time: 9.14e-05 s
File: C:\Users\Tom\AppData\Local\Temp\ipykernel_23956\1118029139.py
Function: attack_ready at line 30

Line #      Hits         Time  Per Hit   % Time  Line Contents
    30                                           def attack_ready(bot, unit: Unit, target: Unit) -> bool:
    31                                               """
    32                                               Determine whether the unit can attack the target by the time the unit faces the target.
    33                                               Thanks Sasha for writing this out.
    34                                               """
    35                                               # fix for units, where this method returns False so the unit moves
    36                                               # but the attack animation is still active, so the move command cancels the attack
    37                                               # need to think of a better fix 

In [51]:
%%cython

from sc2.ids.unit_typeid import UnitTypeId as UnitID
from sc2.unit import Unit
from ares.dicts.turn_rate import TURN_RATE
from ares.dicts.unit_data import UNIT_DATA
UNIT_DATA_INT_KEYS = {k.value: v for k, v in UNIT_DATA.items()}
TURN_RATE_INT_KEYS = {k.value: v for k, v in TURN_RATE.items()}
import math
from libc.math cimport atan2, fabs, pi

cdef double cy_distance_to(
        (float, float) p1,
        (float, float) p2
):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

cdef double angle_to((float, float) from_pos, (float, float) to_pos):
    """Angle from point to other point in radians"""
    return atan2(to_pos[1] - from_pos[1], to_pos[0] - to_pos[0])

cdef double angle_diff(double a, double b):
    """Absolute angle difference between 2 angles"""
    if a < 0:
        a += pi * 2
    if b < 0:
        b += pi * 2
    return fabs(a - b)

cdef double get_turn_speed(unit, unsigned int unit_type_int):
    """Returns turn speed of unit in radians"""
    cdef double turn_rate

    turn_rate = TURN_RATE_INT_KEYS.get(unit_type_int, None)
    if turn_rate:
        return turn_rate * 1.4 * pi / 180

cdef double range_vs_target(unit, target):
    """Get the range of a unit to a target."""
    if unit.can_attack_air and target.is_flying:
        return unit.air_range
    else:
        return unit.ground_range

cpdef bint cy_attack_ready(bot, unit, target):
    """
    Determine whether the unit can attack the target by the time the unit faces the target.
    Thanks Sasha for writing this out.
    """
    cdef:
        unsigned int unit_type_int = unit._proto.unit_type
        unsigned int weapon_cooldown
        double angle, distance, move_time, step_time, turn_time, unit_speed
        (float, float) unit_pos
        (float, float) target_pos

    # fix for units, where this method returns False so the unit moves
    # but the attack animation is still active, so the move command cancels the attack
    # need to think of a better fix later, but this is better then a unit not attacking
    # and still better then using simple weapon.cooldown == 0 micro
    weapon_cooldown = unit.weapon_cooldown
    if weapon_cooldown > 7 and unit_type_int == 91:  # 91 == UnitID.HYDRALISK
        return True
    # prevents crash, since unit can't move
    if unit_type_int == 91:  # == UnitID.LURKERMPBURROWED
        return True
    if not unit.can_attack:
        return False
    # Time elapsed per game step
    step_time = bot.client.game_step / 22.4

    unit_pos = unit.position
    target_pos = target.position
    # Time it will take for unit to turn to face target
    angle = angle_diff(
        unit.facing, angle_to(unit_pos, target_pos)
    )
    turn_time = angle / get_turn_speed(unit, unit_type_int)

    # Time it will take for unit to move in range of target
    distance = (
        cy_distance_to(unit_pos, target_pos)
        - unit.radius
        - target.radius
        - range_vs_target(unit, target)
    )
    distance = max(0, distance)
    unit_speed = (unit.real_speed + 1e-16) * 1.4
    move_time = distance / unit_speed

    return step_time + turn_time + move_time >= weapon_cooldown / 22.4

In [52]:
%timeit cy_attack_ready(bot, unit, target)

1.42 µs ± 8.85 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [53]:
cy_attack_ready(bot, unit, target)

True

# Pick enemy target

In [54]:
def pick_enemy_target(enemies: Units) -> Unit:
    """For best enemy target from the provided enemies
    TODO: If there are multiple units that can be killed, pick the highest value one
        Unit parameter to allow for this in the future

    For now this returns the lowest health enemy
    """
    return min(
        enemies,
        key=lambda e: (e.health + e.shield, e.tag),
    )

In [55]:
pick_enemy_target(units)

Unit(name='KarakMale', tag=4316200961)

In [56]:
%timeit pick_enemy_target(units)

150 µs ± 358 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [57]:
%%cython

cpdef object cy_pick_enemy_target(enemies):
    """For best enemy target from the provided enemies
    TODO: If there are multiple units that can be killed, pick the highest value one
        Unit parameter to allow for this in the future

    For now this returns the lowest health enemy
    """
    cdef:
        object returned_unit
        unsigned int num_enemies, x
        double lowest_health, total_health
    
    num_enemies = len(enemies)
    returned_unit = enemies[0]
    lowest_health = 999.9
    for x in range(num_enemies):
        unit = enemies[x]
        total_health = unit.health + unit.shield
        if total_health < lowest_health:
            lowest_health = total_health
            returned_unit = unit

    return unit

In [58]:
%timeit cy_pick_enemy_target(units)

92.7 µs ± 239 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [59]:
cy_pick_enemy_target(units)

Unit(name='MineralField750', tag=4326162433)

# Converting `units.sorted_by_distance_to`


In [60]:
position = bot.start_location

In [61]:
%timeit units.sorted_by_distance_to(position)

332 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [62]:
units.sorted_by_distance_to(position)[:12]

[Unit(name='CommandCenter', tag=4347396097),
 Unit(name='SCV', tag=4349231105),
 Unit(name='SCV', tag=4348968961),
 Unit(name='SCV', tag=4349755393),
 Unit(name='SCV', tag=4349493249),
 Unit(name='SCV', tag=4348706817),
 Unit(name='SCV', tag=4348444673),
 Unit(name='SCV', tag=4350279681),
 Unit(name='SCV', tag=4350017537),
 Unit(name='SCV', tag=4348182529),
 Unit(name='SCV', tag=4347920385),
 Unit(name='SCV', tag=4350541825)]

In [63]:
%%cython

import numpy as np
cimport numpy as cnp

cdef double cy_distance_to_squared(
        (float, float) p1,
        (float, float) p2
):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

cpdef object cy_sorted_by_distance_to(object units, (float, float) position, bint reverse=False):
    cdef:
        unsigned int len_units = len(units)
        cnp.ndarray[cnp.npy_double, ndim=1] distances = np.empty(len_units)
        # couldn't get this to work
        # cnp.ndarray[cnp.npy_double, ndim=1] indices = np.empty(len_units)
        unsigned int i, j
 
    for i in range(len_units):
        distances[i] = cy_distance_to_squared(units[i].position, position)

    indices = distances.argsort()

    return [units[j] for j in indices]

In [64]:
cy_sorted_by_distance_to(units, position)[:12]

[Unit(name='CommandCenter', tag=4347396097),
 Unit(name='SCV', tag=4349231105),
 Unit(name='SCV', tag=4348968961),
 Unit(name='SCV', tag=4349755393),
 Unit(name='SCV', tag=4349493249),
 Unit(name='SCV', tag=4348706817),
 Unit(name='SCV', tag=4348444673),
 Unit(name='SCV', tag=4350279681),
 Unit(name='SCV', tag=4350017537),
 Unit(name='SCV', tag=4348182529),
 Unit(name='SCV', tag=4347920385),
 Unit(name='SCV', tag=4350541825)]

In [65]:
%timeit cy_sorted_by_distance_to(units, position)

38.1 µs ± 344 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Convert `unit.is_facing()`

In [66]:
my_unit = bot.workers[0]
other_unit = bot.townhalls[0]

In [67]:
%timeit my_unit.is_facing(other_unit)

3.06 µs ± 14.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [68]:
%%cython

from libc.math cimport atan2, fabs, pi



import numpy as np



cpdef bint cy_is_facing(unit, other_unit, double angle_error=0.05):
    cdef:
        (double, double) p1 = unit.position
        (double, double) p2 = other_unit.position
        double angle, angle_difference
        double unit_facing = unit.facing

    angle = atan2(
        p2[1] - p1[1],
        p2[0] - p1[0],
    )
    if angle < 0:
        angle += pi * 2
    angle_difference = fabs(angle - unit_facing)
    return angle_difference < angle_error

In [69]:
%timeit cy_is_facing(my_unit, other_unit)

320 ns ± 3.32 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Convert `Point2.towards()`

In [70]:
pos = bot.game_info.map_center
towards_pos = bot.start_location

In [71]:
%timeit pos.towards(towards_pos, 10.0)

2.79 µs ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [72]:
pos.towards(towards_pos, 10.0)

(108.41400663899812, 76.59588191478983)

In [73]:
pos.towards(towards_pos, -10.0)

(91.58599336100188, 87.40411808521017)

In [74]:
%%cython

from libc.math cimport sqrt
from sc2.position import Point2

cpdef (double, double) cy_towards((double, double) start_pos, (double, double) target_pos, double distance):
    cdef:
        (double, double) vector, displacement, new_pos, normalized_vector
        double magnitude 

    # Calculate the vector between the points
    vector = (target_pos[0] - start_pos[0], target_pos[1] - start_pos[1])

    # Normalize the vector
    magnitude = sqrt(vector[0]**2 + vector[1]**2)
    normalized_vector = (vector[0] / magnitude, vector[1] / magnitude)

    # Calculate the displacement vector
    displacement = (normalized_vector[0] * distance, normalized_vector[1] * distance)

    # Calculate the new position
    new_pos = (start_pos[0] + displacement[0], start_pos[1] + displacement[1])

    return new_pos

In [75]:
%timeit cy_towards(pos, towards_pos, 10.0)

193 ns ± 0.571 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [76]:
cy_towards(pos, towards_pos, 10.0)

(108.41400663899812, 76.59588191478983)

In [77]:
cy_towards(pos, towards_pos, -10.0)

(91.58599336100188, 87.40411808521017)